In [2]:
import os,sys
import subprocess
import threading

working_path="./simhome/"

SIM_BIN="./bin/sim-outorder"
CONFIG="-config ./base2.txt"

parameters = ""
stats_output_path="./configs/nottaken/Stats_dijkstra.txt"
app_bin="./apps/dijkstra/dijkstra_small"
app_parameters="./apps/dijkstra/input.dat"


def run_a_cmd(cmd, is_print=False):
  # print(cmd, flush=True)
  cmd = "cd " + working_path + ";" + cmd
  result = subprocess.run(cmd,shell=True)
  if is_print==True:
    print("done cmd", flush= True)


In [3]:
# clean all old results
run_a_cmd("rm -r ./configs/task_1")

predictor_list=[
    "nottaken", 
    "perfect", 
    "bimod", 
    "2lev", 
    "comb"
    ]

predictor_parameter_map = {
  "nottaken":[""],
  "perfect":[""],
  "bimod": ["-bpred:bimod 1024", "-bpred:bimod 2048"],
  # 2-level predictor config (<l1size> <l2size> <hist_size> <xor>)
  # -bpred:2lev            1 1024 8 0   
  "2lev": ["-bpred:2lev 1 1024 8 0",   # default setting
           "-bpred:2lev 2 1024 8 0",   # doublling  L1 size
           "-bpred:2lev 1 2048 8 0",   # doublling  L2 size
           "-bpred:2lev 2 2048 8 0",   # doublling L1 and L2 size
          ],
  # combining predictor config (<meta_table_size>)
  # -bpred:comb            1024
  "comb": [ "-bpred:comb 1024", # default setting 
            "-bpred:comb 2048", # doublling table size
          ]
}

app_map = {
  # $DIR/Stats_dijkstra.txt ${SHOME}/apps/dijkstra/dijkstra_small  ${SHOME}/apps/dijkstra/input.dat
  "dijkstra_small": "./apps/dijkstra/dijkstra_small ./apps/dijkstra/input.dat > /dev/null",
  # ${DIR}/Stats_stringsearch-cabce.txt ${SHOME}/apps/stringsearch/search_small  &
  "stringsearch-cabce": "./apps/stringsearch/search_small > /dev/null",
  # ${SHOME}/apps/qsort/qsort_small  ${SHOME}/apps/qsort/input_small.dat
  "qsort_small": "./apps/qsort/qsort_small ./apps/qsort/input_small.dat > /dev/null",
  # ${SHOME}/apps/gsm-untoast/untoast -fps -c ${SHOME}/apps/gsm-untoast/small.au.run.gsm
  "gsm-untoast": "./apps/gsm-untoast/untoast -fps -c ./apps/gsm-untoast/small.au.run.gsm > /dev/null",
  # ${SHOME}/apps/jpeg-cjpeg/cjpeg -dct int -progressive -opt -outfile ${SHOME}/configs/${DIR}/jpeg-cjpeg_output_small_encode.jpeg ${SHOME}/apps/jpeg-cjpeg/input_small.ppm
  "jpeg-cjpeg": "./apps/jpeg-cjpeg/cjpeg -dct int -progressive -opt -outfile /tmp/jpeg-cjpeg_output_small_encode.jpeg ./apps/jpeg-cjpeg/input_small.ppm > /dev/null"
}

print(app_map.keys())
plist = []
for predictor in predictor_list:
  predictor_parameter_list = predictor_parameter_map[predictor]
  for predictor_prarmeter in predictor_parameter_list:
    output_folder = predictor
    if predictor_prarmeter != "":
      output_folder = "_".join([predictor,"_".join(predictor_prarmeter.split(" ")[1:])])
    
    output_folder = os.path.join("./configs/task_1/", output_folder)
    run_a_cmd("mkdir -p " + output_folder)
    for app in app_map.keys(): 
      stats_output_path=os.path.join(output_folder,"Stats_" + app + ".txt" )
      cmd= SIM_BIN + " "  + CONFIG + " "\
          + "-bpred " + predictor + " " + predictor_prarmeter \
          + " -redir:sim " + stats_output_path \
          + " "  + app_map[app]
      t = threading.Thread(target=run_a_cmd,args=(cmd,))
      # run_a_cmd(cmd, True)
      t.start()
      plist.append(t)
    # print(output_folder)
print("%d thread have been strated!, waitting them to be done!"% len(plist))
# wait all thread done
for i in range(len(plist)):
  t= plist[i]
  t.join()
  print("%d:%d done!"%(i+1, len(plist)))


dict_keys(['dijkstra_small', 'stringsearch-cabce', 'qsort_small', 'gsm-untoast', 'jpeg-cjpeg'])
5 thread have been strated!, waitting them to be done!
1:5 done!
2:5 done!
3:5 done!
4:5 done!
5:5 done!
